In [3]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.8.5'

In [4]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [5]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [22]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

In [16]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2020-12-16 13:45:02 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 13:45:02 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  4 2020, 00:03:40) [MSC v.1916 32 bit (Intel)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1, Platform Windows-10-10.0.19041-SP0
2020-12-16 13:45:02 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-12-16 13:45:02 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2020-12-16 13:45:02 [py.warnings] WARNING: C:\Users\Gabe\Anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)



ReactorNotRestartable: 

In [21]:
import pandas as pd
dfjson = pd.read_json('quoteresult.json')
dfjson

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,“This life is what you make it. No matter what...,Marilyn Monroe,"[friends, heartbreak, inspirational, life, lov..."
2,“It takes a great deal of bravery to stand up ...,J.K. Rowling,"[courage, friends]"
3,"“If you can't explain it to a six year old, yo...",Albert Einstein,"[simplicity, understand]"
4,"“You may not be her first, her last, or her on...",Bob Marley,[love]
5,"“I like nonsense, it wakes up the brain cells....",Dr. Seuss,[fantasy]
6,"“I may not have gone where I intended to go, b...",Douglas Adams,"[life, navigation]"
7,"“The opposite of love is not hate, it's indiff...",Elie Wiesel,"[activism, apathy, hate, indifference, inspira..."
8,"“It is not a lack of love, but a lack of frien...",Friedrich Nietzsche,"[friendship, lack-of-friendship, lack-of-love,..."
9,"“Good friends, good books, and a sleepy consci...",Mark Twain,"[books, contentment, friends, friendship, life]"


In [11]:
dfjl = pd.read_json('quoteresult.jl', lines=True)
dfjl

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,“This life is what you make it. No matter what...,Marilyn Monroe,"[friends, heartbreak, inspirational, life, lov..."
2,“It takes a great deal of bravery to stand up ...,J.K. Rowling,"[courage, friends]"
3,"“If you can't explain it to a six year old, yo...",Albert Einstein,"[simplicity, understand]"
4,"“You may not be her first, her last, or her on...",Bob Marley,[love]
5,"“I like nonsense, it wakes up the brain cells....",Dr. Seuss,[fantasy]
6,"“I may not have gone where I intended to go, b...",Douglas Adams,"[life, navigation]"
7,"“The opposite of love is not hate, it's indiff...",Elie Wiesel,"[activism, apathy, hate, indifference, inspira..."
8,"“It is not a lack of love, but a lack of frien...",Friedrich Nietzsche,"[friendship, lack-of-friendship, lack-of-love,..."
9,"“Good friends, good books, and a sleepy consci...",Mark Twain,"[books, contentment, friends, friendship, life]"


In [12]:
dfjson.to_pickle('quotejson.pickle')
dfjl.to_pickle('quotejl.pickle')